In [1]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import random
import re


StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 3, Finished, Available, Finished)

In [2]:

# Mount external lakehouse  "Bronze" into specified local path.
notebookutils.fs.mount("abfss://8c7d2edc-a6a8-4f1a-bef1-c8b14449cfd3@onelake.dfs.fabric.microsoft.com/a1b6b3d0-9bef-4e2a-a731-81e4d801d7a1", "/lakehouse/Bronze")

import pandas as pd
# Load data into pandas DataFrame from notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/Juegos en steam.parquet')
df = pd.read_parquet(notebookutils.fs.getMountPath('/lakehouse/Bronze/Files/Data_Parquet/Juegos en steam.parquet'))
display(df)


StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 11253437-68bd-4297-a0a9-c1dcd452471c)

SynapseWidget(Synapse.DataFrame, 1a43dfec-0938-41cd-874e-043072bd0979)

In [3]:
def data_summ(df, title=None):
    '''
    Function to provide detailed information about the dtype, null values,
    and outliers for each column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame for which information is to be generated.
    - title (str, optional): Title to be used in the summary. If None, the title will be omitted.

    Returns:
    - df_info (pd.DataFrame): A DataFrame containing information about each column,
                              including data type, non-missing quantity, percentage of
                              missing values, missing quantity, and information about outliers.
    '''
    info_dict = {"Column": [], "Data_type": [], "No_miss_Qty": [], "%Missing": [], "Missing_Qty": []}

    for column in df.columns:
        info_dict["Column"].append(column)
        info_dict["Data_type"].append(df[column].apply(type).unique())
        info_dict["No_miss_Qty"].append(df[column].count())
        info_dict["%Missing"].append(round(df[column].isnull().sum() * 100 / len(df), 2))
        info_dict['Missing_Qty'].append(df[column].isnull().sum())

  
    df_info = pd.DataFrame(info_dict)

    if title:
        print(f"{title} Summary")
        print("\nTotal rows: ", len(df))
        print("\nTotal full null rows: ", df.isna().all(axis=1).sum())

    return df_info

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 5, Finished, Available, Finished)

In [4]:
data_summ(df, title=None)

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 6, Finished, Available, Finished)

,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,appid,[<class 'int'>],27075,0.00,0
1,name,[<class 'str'>],27075,0.00,0
2,release_date,[<class 'str'>],27075,0.00,0
3,english,[<class 'int'>],27075,0.00,0
4,developer,"[<class 'str'>, <class 'NoneType'>]",27074,0.00,1
5,publisher,"[<class 'str'>, <class 'NoneType'>]",27061,0.05,14
6,platforms,[<class 'str'>],27075,0.00,0
7,required_age,[<class 'int'>],27075,0.00,0
8,categories,[<class 'str'>],27075,0.00,0
9,genres,[<class 'str'>],27075,0.00,0


In [5]:
unique_rating = df['required_age'].unique()
print(unique_rating)

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 7, Finished, Available, Finished)

[ 0 18 16 12  7  3]


In [6]:
# Change the format of the 'release_date' column to just the year.
df['release_date'] = pd.to_datetime(df['release_date']).dt.year

# Change all values ​​in the 'platforms' column to 'PC'.
df['platforms'] = 'PC-STEAM'

# Map the 'required_age' values ​​to the corresponding categories.
age_mapping = {
    0: 'E',
    3: 'E',
    7: 'E10+',
    12: 'T',
    16: 'M',
    18: 'AO'
}
df['required_age'] = df['required_age'].map(age_mapping)

# Define a function to generate random values ​​based on the specified range.
def random_owners(owners_range):
    min_val, max_val = map(int, owners_range.split('-'))
    return random.randint(min_val, max_val)


# Apply the function to the 'owners' column to convert the values ​​and assign random values.
df['owners'] = df['owners'].apply(random_owners)

# Create new column 'Global_Sales' containing multiplication of 'owners' and 'price.
#df['Global_Sales'] = (df['owners'] * df['price']) / 1000000
#df['Global_Sales'] = df['Global_Sales'].round(2)


StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 8, Finished, Available, Finished)

In [7]:
df

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 9, Finished, Available, Finished)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,18713458,7.19
1,20,Team Fortress Classic,1999,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,9394994,3.99
2,30,Day of Defeat,2003,1,Valve,Valve,PC-STEAM,E,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5067102,3.99
3,40,Deathmatch Classic,2001,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,6505582,3.99
4,50,Half-Life: Opposing Force,1999,1,Gearbox Software,Valve,PC-STEAM,E,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5627277,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019,1,SHEN JIAWEI,SHEN JIAWEI,PC-STEAM,E,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,7500,2.09
27071,1065570,Cyber Gun,2019,1,Semyon Maximov,BekkerDev Studio,PC-STEAM,E,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,2712,1.69
27072,1065650,Super Star Blast,2019,1,EntwicklerX,EntwicklerX,PC-STEAM,E,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,9754,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019,1,Yustas Game Studio,Alawar Entertainment,PC-STEAM,E,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,16318,5.19


In [8]:
# Paso 1: Dividir la columna 'genres' en listas
df['genres'] = df['genres'].str.split(';')

# Paso 2: Explode para separar los géneros en filas individuales
df = df.explode('genres')

# Paso 3: Verificar el resultado
df

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 10, Finished, Available, Finished)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,18713458,7.19
1,20,Team Fortress Classic,1999,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,9394994,3.99
2,30,Day of Defeat,2003,1,Valve,Valve,PC-STEAM,E,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5067102,3.99
3,40,Deathmatch Classic,2001,1,Valve,Valve,PC-STEAM,E,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,6505582,3.99
4,50,Half-Life: Opposing Force,1999,1,Gearbox Software,Valve,PC-STEAM,E,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5627277,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,1066700,New Yankee 7: Deer Hunters,2019,1,Yustas Game Studio,Alawar Entertainment,PC-STEAM,E,Single-player;Steam Cloud,Casual,Indie;Casual;Adventure,0,2,0,0,0,16318,5.19
27073,1066700,New Yankee 7: Deer Hunters,2019,1,Yustas Game Studio,Alawar Entertainment,PC-STEAM,E,Single-player;Steam Cloud,Indie,Indie;Casual;Adventure,0,2,0,0,0,16318,5.19
27074,1069460,Rune Lord,2019,1,Adept Studios GD,Alawar Entertainment,PC-STEAM,E,Single-player;Steam Cloud,Adventure,Indie;Casual;Adventure,0,4,0,0,0,14192,5.19
27074,1069460,Rune Lord,2019,1,Adept Studios GD,Alawar Entertainment,PC-STEAM,E,Single-player;Steam Cloud,Casual,Indie;Casual;Adventure,0,4,0,0,0,14192,5.19


In [9]:
new_names = {'appid': 'Item_id', 'name': 'Item_name', 'release_date': 'Release', 'genres': 'Genres', 'developer': 'Developer', 'publisher': 'Publisher', 'platforms': 'Platform', 'required_age': 'Rating', 'average_playtime': 'Average_playtime', 'owners': 'Owners', 'price': 'Price' }

# Use the rename() method to change column names
df.rename(columns=new_names, inplace=True)

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 11, Finished, Available, Finished)

In [10]:
column_order = ['Item_id', 'Item_name', 'Platform', 'Genres', 'Release', 'Developer', 'Publisher',  'Rating', 'positive_ratings', 'negative_ratings', 'Average_playtime', 'Owners', 'Price']
df = df[column_order]
df.head()

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 12, Finished, Available, Finished)

,Item_id,Item_name,Platform,Genres,Release,Developer,Publisher,Rating,positive_ratings,negative_ratings,Average_playtime,Owners,Price
0,10,Counter-Strike,PC-STEAM,Action,2000,Valve,Valve,E,124534,3339,17612,18713458,7.19
1,20,Team Fortress Classic,PC-STEAM,Action,1999,Valve,Valve,E,3318,633,277,9394994,3.99
2,30,Day of Defeat,PC-STEAM,Action,2003,Valve,Valve,E,3416,398,187,5067102,3.99
3,40,Deathmatch Classic,PC-STEAM,Action,2001,Valve,Valve,E,1273,267,258,6505582,3.99
4,50,Half-Life: Opposing Force,PC-STEAM,Action,1999,Gearbox Software,Valve,E,5250,288,624,5627277,3.99


In [12]:
archivo_parquet = '/lakehouse/default/Files/Cleaned_Data/games_steam.parquet'

# Export to a parquet file.
df.to_parquet(archivo_parquet, index=False)

StatementMeta(, 4dec1f18-246e-4ed3-9c68-390657978dbc, 14, Finished, Available, Finished)